In [2]:
%%capture
!pip install altair transformers accelerate bitsandbytes sentencepiece protobuf tabulate
!pip install git+https://github.com/run-llama/llama_index

In [18]:
import pandas as pd
import sqlite3

db_path = 'Employees.db'
conn = sqlite3.connect(db_path)
c = conn.cursor()
def sq(q):
    with conn:
        df = pd.read_sql_query(q, conn)
        return df
    
data = [
    ["""What are the names and job titles of active employees in the IT department?""",
     """SELECT e.firstname, e.lastname, e.title FROM employee_data e WHERE e.departmenttype = 'IT/IS' AND e.employeestatus = 'Active' ORDER BY e.firstname, e.lastname NULLS LAST;"""],
    ["""What are the names of employees who have a performance score of 'Needs Improvement'?""",
     """SELECT e.firstname, e.lastname FROM employee_data e WHERE e.performance_score = 'Needs Improvement';"""],
    ["""What is the average years of experience for applicants in each education level category?""",
     """SELECT r.education_level, AVG(r.years_of_experience) AS average_years_of_experience FROM recruitment_data r GROUP BY r.education_level ORDER BY average_years_of_experience DESC NULLS LAST;"""],
    ["""Calculate the total training cost for each location and display the results in descending order of total cost""",
     """SELECT t.location, SUM(t.training_cost) AS total_training_cost FROM training_and_development_data t GROUP BY t.location ORDER BY total_training_cost DESC;"""],
    ["""What are the names and total training costs for employees who have participated in both the "Customer Service" and "Technical Skills" training programs?""",
     """SELECT e.firstname, e.lastname, SUM(t.training_cost) AS total_training_cost FROM training_and_development_data t JOIN employee_data e ON t.employee_id = e.employee_id WHERE t.training_program_name IN ('Customer Service', 'Technical Skills') GROUP BY e.firstname, e.lastname ORDER BY total_training_cost DESC NULLS LAST;"""],
    ["What are the top 3 trainers based on the total number of employees they have provided training to?",
     """SELECT t.trainer, COUNT(t.employee_id) AS total_employees FROM training_and_development_data t GROUP BY t.trainer ORDER BY total_employees DESC NULLS LAST LIMIT 3;"""]
    ]

eval_df = pd.DataFrame({"question": [d[0] for d in data], 'query': [d[1] for d in data], 'data': [sq(d[1]).to_dict(orient='list') for d in data]})

# eval_df.to_csv('data/eval_data.csv')

# path = '/kaggle/input/temp-alaa-zip/'
# large_data = pd.read_csv(path+'large_table.csv')
# large_data_hard = pd.read_csv(path+'large_table_hard.csv', index_col=0)
# medium_data = pd.read_csv(path+'medium_table.csv', index_col=0)
# small_data = pd.read_csv(path+'small_table.csv', index_col=0)

# question_small = """What is the employee count for each race in each department in the dataset?"""
# question_medium = """What is the average engagement score for males and females for each title in the dataset?"""
# question_large = """What is the employees count and	avgerage training duration for each location in the dataset?"""
# question_large_hard = """For each combination of education level and job function description in recruitment data, how many distinct applicants are there with more than 5 years of experience?"""

# eval_df = pd.read_csv('/kaggle/input/eval-viz/eval_data.csv', index_col=0)

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
from llama_index.llms.huggingface import (
  HuggingFaceLLM, PromptTemplate
)
from transformers import BitsAndBytesConfig
import torch

model_name = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     load_in_8bit_fp32_cpu_offload=True
#     )

model = HuggingFaceLLM(
    model_name = model_name,
    tokenizer = tokenizer,
    query_wrapper_prompt = PromptTemplate("{query_str}"),
    # context_window=3900,
    max_new_tokens=1024,
#     model_kwargs = {"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs = {
        "num_return_sequences":1,
        "eos_token_id":tokenizer.eos_token_id,
        "pad_token_id":tokenizer.eos_token_id,
        "do_sample":False,
        "num_beams":1
        },
    device_map = "auto",
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /opt/conda/lib/python3.10/site-
[nltk_data]     packages/llama_index/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /opt/conda/lib/python3.10/site-
[nltk_data]     packages/llama_index/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
def inference(prompt: str):
    response = model.complete(prompt).text.strip()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    return response

In [7]:
prompt_vega = '''
Generate an insightful Vega-Lite visualization (Bar Chart, Pie Chart, Line Chart, etc.) that effectively illustrates the data and addresses the user's query.
Ensure the Vega-Lite specification accurately represents the provided data and makes the visualization intuitive and informative.
Pay meticulous attention to the specification's structure, including encoding, mark type, and any data transformations.
Add a clear "title" to the visualization based on the data insights and the question.
Generate only ONE proper json string, no text. Do NOT involve multiple charts such as "hconcat", "vconcat", "layer", "facet", "repeat", "concat", or "resolve".

Consider the following guidelines for selecting the appropriate chart type:
- For categorical data with distinct categories and proportions (Percentages), consider using a Pie Chart for better representation.
- For numerical data with comparisons or trends over time, a Line Chart or Bar Chart might be more suitable.
- If the data involves showing proportions or percentages of a whole, a Pie Chart could be effective.
- For showing distributions or comparisons across categories, a Bar Chart might provide clearer insights.

Question: {question}

Data Sample: {data}

Vega-Lite Json:
'''

In [94]:
import altair as alt
# from altair.vegalite.v5.api import Chart
import pandas as pd
import re
import json

def extract_json(text: str) -> dict:
    """Extracts a valid JSON object from a string.

    Args:
        text: The string to extract the JSON object from.

    Returns:
        The extracted JSON object as a dictionary, or None if no valid JSON
        object is found.
    """

    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass

    # Extract the JSON part from the string if initial parsing fails
    start_index = text.find('{')
    end_index = text.rfind('}') + 1

    if start_index >= 0 and end_index > start_index:
        json_string = text[start_index:end_index]
        try:
            return json.loads(json_string)
        except json.JSONDecodeError:
            pass

    return None


def prepare_vega(
    vega_lite_spec: str,
    data: pd.DataFrame = None,
    width: int = 600,
    height: int = 400,
    tooltip: bool = True,
    max_bars: int = 25,
    top_bars: int = 15
):# -> Chart
    """Creates a Vega-Lite chart from a dictionary containing the specification.

    Args:
        vega_lite_spec_dict: A dictionary containing the Vega-Lite JSON specification.
        data: (Optional) A pandas DataFrame containing the data for the chart.
        width: (Optional) The width of the chart in pixels.
        height: (Optional) The height of the chart in pixels.
        tooltip: (Optional) Whether to include a tooltip.
        max_bars: (Optional) The maximum number of bars to display.
        top_bars: (Optional) The number of top bars to display if exceeding max_bars.

    Returns:
        An Altair Chart object.
    """

    vega_lite_spec_dict = extract_json(vega_lite_spec)

    if vega_lite_spec_dict is None:
        return "Invalid Vega-Lite JSON. Please provide a valid JSON string"

    # try:
    # vega_lite_spec_dict = json.loads(vega_lite_spec)

    # Check if it's a bar plot or pie plot
    is_bar_plot = vega_lite_spec_dict["mark"] == "bar"
    is_pie_chart = vega_lite_spec_dict["mark"] == "arc"

    # Set the schema
    vega_lite_spec_dict["$schema"] = "https://vega.github.io/schema/vega-lite/v5.json"

    # Set width and height
    vega_lite_spec_dict["width"] = width
    vega_lite_spec_dict["height"] = height

    if data is not None:
        vega_lite_spec_dict["data"]["values"] = data.to_dict(orient='records')

        if "mark" in vega_lite_spec_dict:
            # if bar plot
            if vega_lite_spec_dict["mark"] == "bar":
                for var in vega_lite_spec_dict['encoding'].keys():

                    # skipp the tooltip
                    if var == 'tooltip':
                        continue

                    column = vega_lite_spec_dict['encoding'][var]['field']
                    # Check if the column exists in the data
                    if column not in data.columns:
                        continue

                    uniques = data[column].unique().tolist()
                    dtype = data[column].dtype

                    # Set scale domain for non-numeric columns
                    if not pd.api.types.is_numeric_dtype(dtype):
                        try:
                            vega_lite_spec_dict['encoding'][var]['scale']['domain'] = uniques
                        except:
                            pass

                        # select top bars if exceeded the max bars
                        if var == 'x' and len(uniques) >= max_bars:
                            vega_lite_spec_dict['encoding'][var]["sort"] = "-y"
                            # Add a window transform to select only top k bars
                            vega_lite_spec_dict["transform"] = [
                                {
                                    "window": [{"op": "row_number", "as": "rank"}],
                                    "sort": [{"field": "-y", "order": "descending"}]
                                },
                                {
                                    "filter": {"field": "rank", "lte": max_bars}
                                }
                            ]
                            vega_lite_spec_dict["encoding"]["x"]["axis"] = {
                                    "labelAngle": 45
                                }
                    else:
                        try:
                            # Remove scale domain for numeric columns
                            del vega_lite_spec_dict['encoding'][var]['scale']['domain']
                        except:
                            pass

            # Configure tooltip
            if tooltip:
                try:
                    columns = data.columns
                    vega_lite_spec_dict.setdefault("encoding", {})["tooltip"] = [{"field": field, "type": "nominal"} for field in columns]
                except:
                    pass

        elif is_pie_chart:
            pass

#     try:
#         vega_lite_spec_str = json.dumps(vega_lite_spec_dict)
#         return vega_lite_spec_str
#     except ValueError:
#         None
        
    try:
        vega_lite_spec_str = json.dumps(vega_lite_spec_dict)
        chart = alt.Chart.from_json(vega_lite_spec_str).interactive()
    except ValueError:
        vega_lite_spec_str = json.dumps(vega_lite_spec_dict)
        chart = alt.Chart.from_json(vega_lite_spec_str)
    except Exception as e:
        chart = None

    return chart

In [26]:
responses = []
for question, data in eval_df[['question', 'data']].values:
    data = pd.DataFrame(eval(data))
    print(question); print(); print(data); print(); print()
    responses.append(inference(
        prompt_vega.format(
            question=question, 
            data=data.sample(min(5, len(data))),
        )
    ))
    print(responses[-1]); print(); print()

What are the names and job titles of active employees in the IT department?

     firstname  lastname                     title
0         Abby  Woodward                IT Support
1     Abdullah  Arellano           IT Manager - DB
2         Abel     Baker  Principal Data Architect
3         Abel      Bush               IT Director
4       Adison    Huerta              BI Developer
..         ...       ...                       ...
370    Yesenia   Nielsen                IT Support
371    Zackery      Hess        Area Sales Manager
372    Zaniyah     Parks           IT Manager - DB
373       Zara     Pitts          Network Engineer
374  Zechariah    Strong                IT Support

[375 rows x 3 columns]


{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "What are the names and job titles of active employees in the IT department?",
  "data": {
    "values": [
      {
        "firstname": "Matthew",
        "lastname": "Westinghouse",
        "title": "B

In [29]:
eval_df['vega'] = responses

In [31]:
prepare_vega(eval_df['vega'].values[0], pd.DataFrame(eval(eval_df['data'].values[0])))

alt.Chart(...)

In [32]:
prepare_vega(eval_df['vega'].values[1], pd.DataFrame(eval(eval_df['data'].values[1])))

In [44]:
prepare_vega(eval_df['vega'].values[2], pd.DataFrame(eval(eval_df['data'].values[2])))

alt.Chart(...)

In [45]:
prepare_vega(eval_df['vega'].values[3], pd.DataFrame(eval(eval_df['data'].values[3])))

alt.Chart(...)

In [46]:
prepare_vega(eval_df['vega'].values[4], pd.DataFrame(eval(eval_df['data'].values[4])))

alt.Chart(...)

In [118]:
import altair as alt
import pandas as pd
import json

# Provided JSON data
json_data = '''{
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "description": "Top 3 trainers based on the total number of employees they have provided training to",
    "data": {
        "values": [
            {"trainer": "Michael Smith", "total_employees": 4},
            {"trainer": "William Smith", "total_employees": 3},
            {"trainer": "Michael Brown", "total_employees": 3}
        ]
    },
    "mark": "bar",
    "encoding": {
        "x": {"field": "trainer", "type": "nominal"},
        "y": {"field": "total_employees", "type": "quantitative"}
    }
}'''

# Load JSON data into a dictionary
json_dict = json.loads(json_data)

# Load data values into a pandas DataFrame
data = pd.DataFrame(json_dict['data']['values'])

# Create Altair chart
chart = alt.Chart(data).mark_bar().encode(
    x='trainer:N',
    y='total_employees:Q',
    tooltip=['trainer:N', 'total_employees:Q']
).properties(
    width=600,
    height=400
)

# Show the chart
chart

alt.Chart(...)

In [64]:
prepare_vega(eval_df['vega'].values[5], pd.DataFrame(eval(eval_df['data'].values[5])))

,trainer,total_employees
0,Michael Smith,4
1,William Smith,3
2,Michael Brown,3


In [58]:
pd.DataFrame(eval(eval_df['data'][5]))

,trainer,total_employees
0,Michael Smith,4
1,William Smith,3
2,Michael Brown,3


In [120]:
from IPython.display import display, HTML

csv_filename = 'eval_df.csv'
eval_df.to_csv(csv_filename, index=False)
display(HTML(f'<a href="{csv_filename}" target="_blank">Click here to download CSV file</a>'))